In [1]:
import glob
import os
from itertools import combinations

import numpy as np
np.random.seed(18012023)
import pandas as pd
import scipy.spatial.distance as scidist
import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import networkx as nx

In [2]:
length = 3000
titles, texts = [], []
for fn in sorted(glob.glob('../data/texts/rich/*.tsv')):
    title = os.path.basename(fn).replace('.tsv', '')
    with open(fn) as f:
        lines = f.readlines()[1:]
        words = []
        for line in lines:
            line = line.strip()
            if line:
                comps = line.split()
                if len(comps) == 3:
                    _, lem, pos = comps
                    for l, p in zip(lem.split('+'), pos.split('+')):
                        if p != 'n(prop)':
                            words.append(l)
    si, ei = 0, length
    while ei < len(words):
        titles.append(title)
        texts.append(' '.join(words[si:ei]))
        si += length
        ei += length
        
df = pd.DataFrame(zip(titles, texts), columns=('title', 'text'))

In [3]:
df

,title,text
0,"Der naturen bloeme, handschrift D",in jaar ons heer als men schrijven 1287 zijn p...
1,"Der naturen bloeme, handschrift D",voort horen van elk wonder alleen van die zegg...
2,"Der naturen bloeme, handschrift D",er omdat menig dier in staan daar ik dat duits...
3,"Der naturen bloeme, handschrift D",sparen het zijn met alrend zede dat zij ontzui...
4,"Der naturen bloeme, handschrift D",luchter zijde en daarom hij zoeken vlucht de l...
...,...,...
1622,wrake_van_ragisel,doen menig verdriet en worsen en gebaren haar ...
1623,wrake_van_ragisel,zeggen gij dat men daar zeer strijden beide va...
1624,wrake_van_ragisel,vijf jaar het komen zo fierlijk varen met alga...
1625,wrake_van_ragisel,zoeken zaan daarnaar de jonkvrouw al dat land ...


In [4]:
titles = sorted(df['title'].unique())
titles

['Der naturen bloeme, handschrift D',
 'Enaamse Codex',
 'Ferguut',
 'Karel ende Elegast',
 'Lantsloot van der haghedochte',
 'Luiks Diatessaron',
 'Lutgard K',
 'Perchevael',
 'Reinaerts historie',
 'Reinout van Montalbaen',
 'Rijmbijbel',
 'Roman van Moriaen',
 'Roman van Walewein',
 'Sinte Kerstine',
 'Van Sente Brandane',
 'Van den vos Reynaerde',
 'alexanders_geesten',
 'antwerps_liedboek',
 'arturs_doet',
 'beatrijs',
 'bediedenisse_van_der_missen',
 'berijmd_commentaar_op_het_hooglied_fragm_w',
 'berlijnse_liederenhandschrift',
 'boec_van_den_houte',
 'borchgrave_van_couchi',
 'borchgravinne_van_vergi__1',
 'brabantsche_yeesten__1-5',
 'brabantsche_yeesten__6',
 'brabantsche_yeesten__7',
 'cyromanchie_van_den_pape_van_den_hamme_chiromantie',
 'dat_boec_vander_wraken',
 'de_spieghel_der_menscheliker_behoudenesse',
 'der_leken_spieghel',
 'der_mannen_ende_vrouwen_heimelijcheit',
 'der_minnen_loep',
 'der_vrouwen_heimelijcheit',
 'der_ystorien_bloeme',
 'devoot_ende_profitelyck_boe

In [5]:
vec = TfidfVectorizer(min_df=2, token_pattern=r"(?u)\b\w+\b")
X = vec.fit_transform(df['text']).toarray()
X.shape

(1627, 16243)

In [6]:
distances = np.zeros((len(titles), len(titles)))
for t1, t2 in tqdm.tqdm(list(combinations(titles, 2))):
    A = X[df['title'] == t1]
    B = X[df['title'] == t2]
    distance = scidist.cdist(A, B, metric='cosine').mean()
    distances[titles.index(t1), titles.index(t2)] = distance
    distances[titles.index(t2), titles.index(t1)] = distance

100%|██████████| 6786/6786 [00:37<00:00, 180.65it/s]


In [7]:
G = nx.Graph() 
for t1, v in zip(titles, distances):
    tops = v.argsort()[1:4] # ignore self-distance; take top-3 (like Eder)
    for t2 in tops:
        t2 = titles[t2]
        if G.has_edge(t1, t2):
            G[t1][t2]['weight'] += 1
        else:
            G.add_edge(t1, t2, weight=1)
print(f"N nodes = {G.number_of_nodes()}, N edges = {G.number_of_edges()}")
nx.write_gexf(G, "../output/R.gexf")

N nodes = 117, N edges = 312


Caption: 

> Panoramische weergave van 117 Middelnederlandse rijmteksten, in dit netwerk als knooppunten verbonden op basis van hun verwantschap in woordenschat (excl. eigennamen). Deze analyse schaart de Reynaert dicht bij Reinaerts historie, maar ook andere oude Vlaamse werken. Door Mike Kestemont (Universiteit Antwerpen), Creative Commons (CC-BY).

Uitgelichte werken: 
- Reynaert
- Reinaerts historie
- Lantsloot vander haghedochte
- Moriaen
- Karel ende Elegast
- Floris ende Blancefloer
- Roman van Walewein
- Van Sente Brandane

Steps in Gephi:
- Force Atlas (LinLog mode; prevent overlap; dissuade hubs) > wait for rectangular shape
- Data Laboratory > clear Label column, manually enter short titles for focus texts
- Overview: show labels
- Modularity: run
- Nodes > color > partition > modularity class
- Preview:
    - Show labels
    - Font Bebas
- Export as PDF